## Initialization

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))
import torch
from kar_hnn import KARHNN
from data import get_dataset
import numpy as np

## Help Functions

In [2]:
# print squared loss at specific steps for comparison with HNN
print_every = 200
def print_results(results, print_every=200):
    for step in range(0, len(results["train_loss"]), print_every):
        print(
            "step {}, train_loss {:.4e}, test_loss {:.4e}".format(
                step,
                results["train_loss"][step],
                results["test_loss"][step],
            )
        )
    print('Final train loss {:.4e} +/- {:.4e}\nFinal test loss {:.4e} +/- {:.4e}'
        .format(results["train_loss"][-1], results["train_std"][-1],
                results["test_loss"][-1], results["test_std"][-1]))

def print_best(results):
    curr_min = 0

    for step in range(0, len(results["train_loss"])):
        if results["test_loss"][step] < results["test_loss"][curr_min]:
            curr_min = step
    print(
        "best test loss at step {}, train_loss {:.4e}, test_loss {:.4e}".format(
            curr_min,
            results["train_loss"][curr_min],
            results["test_loss"][curr_min],
        )
    )


## Create Dataset

In [3]:
# create dataset
THIS_DIR = os.path.dirname(os.path.abspath(os.path.join(".")))
data = get_dataset("3body", THIS_DIR, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(data["coords"], requires_grad=True, dtype=torch.float32).to(device)
test_x = torch.tensor(data["test_coords"], requires_grad=True, dtype=torch.float32).to(
    device
)
dxdt = torch.Tensor(data["dcoords"]).to(device)
test_dxdt = torch.Tensor(data["test_dcoords"]).to(device)

# dataset['train_input'], dataset['train_label'],dataset['test_input'], dataset['test_label']
dataset = {
    "train_input": x,
    "train_label": dxdt,
    "test_input": test_x,
    "test_label": test_dxdt,
}

Successfully loaded data from /Users/ZongyuWu/hamiltonian-nn/3body-orbits-dataset.pkl


In [4]:
dataset["train_input"].shape, dataset["train_label"].shape

(torch.Size([80000, 12]), torch.Size([80000, 12]))

## Model Training

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# initialize KAN with G=3
model = KARHNN(
    input_dim=3 * 4, width=[3 * 4, 15, 10, 2], grid=2, 
    k=3, seed=42, device=device
)
results = model.fit(dataset, opt="LBFGS", steps=135, log=1, batch=50)
print_results(results, 1)
print_best(results)

cpu
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.29e-03 | test_loss: 1.79e-02 | reg: 6.27e+01 | : 100%|█| 135/135 [00:13<00:00,  9.65

saving model version 0.1
step 0, train_loss 5.2780e-02, test_loss 1.0126e-01
step 1, train_loss 1.2267e-01, test_loss 2.2805e-01
step 2, train_loss 5.1074e-02, test_loss 1.2147e-01
step 3, train_loss 1.3494e-01, test_loss 3.7432e+00
step 4, train_loss 5.1295e-02, test_loss 9.2307e-02
step 5, train_loss 1.8527e-02, test_loss 1.1220e-01
step 6, train_loss 9.3854e-02, test_loss 3.5640e-01
step 7, train_loss 3.0502e-02, test_loss 9.9015e-02
step 8, train_loss 2.7513e-02, test_loss 8.0749e-01
step 9, train_loss 4.0236e+01, test_loss 7.0406e+00
step 10, train_loss 1.7383e-01, test_loss 6.1209e-01
step 11, train_loss 3.9431e-02, test_loss 5.6733e-02
step 12, train_loss 4.3168e-01, test_loss 3.5526e+00
step 13, train_loss 1.3249e-02, test_loss 3.0801e-01
step 14, train_loss 8.6957e-03, test_loss 1.4969e-01
step 15, train_loss 3.6006e-02, test_loss 2.1405e-01
step 16, train_loss 2.1256e-02, test_loss 6.9202e+00
step 17, train_loss 6.5108e-01, test_loss 4.3169e-01
step 18, train_loss 1.3928e+00,

![hnn](./hnn.png)
![baseline](./baseline.png)
